In [22]:
# installing requirements
!pip install nltk
!pip install better-profanity
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from better_profanity import profanity
import pandas as pd

# loading train data
review_df = pd.read_csv(r'C:\code\Anvil\Yhack\review_data.txt', header = 0, sep = "\t", quoting = 3)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rockr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
# making personal information example for testing
post_text="2022.04.09"
address_text = "11, Naesonjungang-ro, Uiwang-si, Gyeonggi-do, Republic of Korea"
number_text="010-xxxx-xxxx"
personal_inform=[post_text, address_text, number_text]



In [24]:
SNS_activity=["2022.04.12","facebook","post", " my name is KYUHWAN CHOI.My number is 010-xxxx-xxxx and arse Today is a really annoying and angry day.. and i hate !"]
print(personal_inform)


['2022.04.09', '11, Naesonjungang-ro, Uiwang-si, Gyeonggi-do, Republic of Korea', '010-xxxx-xxxx']


In [25]:
# porfanity and personal information filter
contents_list=[]
filter_list=[]
contents_list=word_tokenize(SNS_activity[-1])

# profanity_counter = 0
# personal_information_counter = 0

def checking(words_list):
    profanity_counter = 0
    personal_information_counter = 0

    for element in contents_list:
        if profanity.contains_profanity(element) == True:
            profanity_counter += 1
        if element in contents_list:
            personal_information_counter += 1
    if profanity_counter > 0 :
        filter_list.append("profanity")

    if personal_information_counter > 0 :
        filter_list.append("personal information")
    



In [26]:
checking(contents_list)

In [27]:
# Bad mood filter
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 re 모듈을 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

cM_list=[]
cM_list.append(SNS_activity[-1])

In [28]:
# 결정 값 클래스인 sentiment 칼럼을 별도로 추출
from sklearn.model_selection import train_test_split


# 학습 데이터에서 sentiment만 추출
class_df = review_df['sentiment']
# 학습 데이터에서 review만 추출
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.2, random_state=156,shuffle=True)
X_train.shape, X_test.shape



((20040, 1), (5011, 1))

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10, max_iter = 500))]) # C 값이 작으면 훈련을 덜 복잡 ( 강한 규제)

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred =pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}'.format(accuracy_score(y_test ,pred)))

예측 정확도는 0.8773


In [1]:
#pipeline.fit(X_train['review'], y_train)
pred =pipeline.predict(cM_list)
if pred == [0]:
    filter_list.append("bad mood")
#해당 메소드를 실행하면서 학습한 데이터를 기반으로 주어진 Feature가 어떤 확률로 Label로 분류되었는지 계산하여 반환
pipeline.predict_proba(cM_list)

NameError: name 'pipeline' is not defined

In [31]:
# 어떤 확률로 선택했는지 찾는 코드 
percentage1=pipeline.predict_proba(cM_list)[:,0]
percentage2=pipeline.predict_proba(cM_list)[:,1]

def ref(x,y):
    return x if x>=y else y

In [32]:
print("test_sentance: ", SNS_activity[-1],'\n',
     "sentiment :", pred, "\n",
      "percentage:", ref(percentage1,percentage2)*100,"[%]"
     )


test_sentance:   my name is KYUHWAN CHOI.My number is 010-xxxx-xxxx and arse Today is a really annoying and angry day.. and i hate ! 
 sentiment : [0] 
 percentage: [80.42435397] [%]


In [33]:
filter_list

['profanity', 'personal information', 'bad mood']